# Earnings Call Project: WebScraping 
<br>
CIS 831 Deep Learning – Term Project<br>
Kansas State University
<br><br>
James Chapman<br>
John Woods<br>
Nathan Diehl<br>
<br>

This notebook collects various types of stock price data from 2 different APIs (Yahoo & Alphavantage). 
- [yfinance](https://pypi.org/project/yfinance/)
- [Alphavantage Documentation](https://www.alphavantage.co/documentation/)

In the original papers, the target metric is volatility. This is defined as the standard deviation of the dividend-adjusted closing price, measured n_days after the earnings call (3, 7, 15, 30 days). The loss function is the MSE of the volatility. See Data_Cleaning notebook for more.

The data from this notebook is stored in the "data/data_prep" directory as the following CSVs, as well as the raw data from Alphavantage in the "alpha_data" directory.
- yahoo_data
- alpha_data
- MAEC_yahoo_data
- MAEC_alpha_data

In [1]:
import sys
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install yfinance
    !pip install load_dotenv
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd gdrive/My Drive/831

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import time
import os
from datetime import datetime
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv() 

True

In [3]:
MAEC_dir = 'data/MAEC/MAEC_Dataset' # https://github.com/Earnings-Call-Dataset/MAEC-A-Multimodal-Aligned-Earnings-Conference-Call-Dataset-for-Financial-Risk-Prediction

############# too big for GitHub ########################
############# stored on local disk ######################
original_data_dir = r"D:\original_dataset" # https://github.com/GeminiLn/EarningsCall_Dataset 
MAEC_audio_dir = r"D:\MAEC_audio" 
# there is a link for the audio data in the MAEC GitHub, but it does not work
# I emailed the authors, and they send another link.
# There is like a half-million files, but only 19 GB
# https://drive.google.com/file/d/1m1GRCHgKn9Vz9IFMC_SpCog6uP3-gFgY/view?usp=drive_link

# Original dataset
[paper](https://aclanthology.org/P19-1038.pdf)
[GitHub](https://github.com/GeminiLn/EarningsCall_Dataset)

In [4]:
# Loop through the directory, each folder represents an earnings conference call; the folders are named as "CompanyName_Date".
filename_data = []
for filename in os.listdir(original_data_dir):
    company_name, date_str = filename.rsplit('_', 1)
    date_str = date_str.split('.')[0] 
    date = datetime.strptime(date_str, "%Y%m%d").strftime("%Y-%m-%d")
    filename_data.append([company_name, date])
filename_data = pd.DataFrame(filename_data, columns=["Company", "Date"])
filename_data.drop_duplicates(inplace=True)

# Special file I made to match the company name to the ticker
company_ticker = pd.read_csv('data/data_prep/company_ticker.csv')
filename_data = filename_data.merge(company_ticker, on="Company", how="left")
filename_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Company  572 non-null    object
 1   Date     572 non-null    object
 2   Ticker   572 non-null    object
dtypes: object(3)
memory usage: 13.5+ KB


# Yahoo 

In [5]:
tickers = filename_data.Ticker.unique().tolist()
# 26 tickers do not have data on Yahoo! (No longer trading etc.)
tickers_to_remove = ['GGP', 'CA', 'STI', 'FLT', 'NLSN', 'WRK','RTN', 'UTX', 'DISH', 'VIACOM', 
                     'TIF', 'CELG', 'RHT', 'FLIR', 'AGN', 'KSU', 'NBL', 'ALXN', 'ABMD ', 'CTXS', 
                     'CBS', 'XL', 'VAR', 'ATVI', 'XLNX', 'SIVB']
tickers = [ticker for ticker in tickers if ticker not in tickers_to_remove]
yahoo_data = yf.download(tickers, start="2017-01-01", end="2017-12-31", group_by="ticker")
yahoo_data.index = yahoo_data.index.tz_localize(None)
yahoo_data.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in yahoo_data.columns]
yahoo_data = yahoo_data.sort_index(axis=1)
### save ############################################
yahoo_data.to_csv('data/data_prep/yahoo_data.csv', index=True)
#####################################################
yahoo_data.info(verbose=True)

[*********************100%***********************]  254 of 254 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2017-01-03 to 2017-12-29
Data columns (total 1524 columns):
 #     Column           Dtype  
---    ------           -----  
 0     AAP_Adj Close    float64
 1     AAP_Close        float64
 2     AAP_High         float64
 3     AAP_Low          float64
 4     AAP_Open         float64
 5     AAP_Volume       int64  
 6     ABBV_Adj Close   float64
 7     ABBV_Close       float64
 8     ABBV_High        float64
 9     ABBV_Low         float64
 10    ABBV_Open        float64
 11    ABBV_Volume      int64  
 12    ABT_Adj Close    float64
 13    ABT_Close        float64
 14    ABT_High         float64
 15    ABT_Low          float64
 16    ABT_Open         float64
 17    ABT_Volume       int64  
 18    ADBE_Adj Close   float64
 19    ADBE_Close       float64
 20    ADBE_High        float64
 21    ADBE_Low         float64
 22    ADBE_Open        float64
 23    ADBE_Volume      int64  
 24    ADM_Adj Close    float64
 25    ADM

# Alphadvantage

In [6]:
API_KEY = os.getenv('API_KEY') # used with www.alphavantage.co
alpha_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}&outputsize=full&datatype=json' #.format(ticker,API_KEY)
alpha_dir = 'data/alpha_data/{}.csv' #.format(ticker) # I saved the raw alphadvantage data, so I don't have to do it again
alpha_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividend Amount', 'Split Coefficient'] # same as Yahoo

alpha_data = pd.DataFrame()
tickers = filename_data.Ticker.unique().tolist()
for ticker in tqdm(tickers):
    try:
        response = requests.get(alpha_url.format(ticker,API_KEY)).json()
        ticker_df = pd.DataFrame(response['Time Series (Daily)']).T
        ticker_df.columns = [f"{ticker}_{col}" for col in alpha_columns] # same as Yahoo
        ticker_df.index = pd.to_datetime(ticker_df.index)
        ticker_df = ticker_df.apply(pd.to_numeric, errors='coerce')
        ### save ############################################
        ticker_df.to_csv(alpha_dir.format(ticker), index=True) # dates are index
        #####################################################
        ticker_df_2017 = ticker_df[ticker_df.index.year == 2017]
        ticker_df_2017 = ticker_df_2017.rename(columns={f"{ticker}_Adj Close": ticker})
        ticker_df_2017 = ticker_df_2017[[ticker]].copy()
        if alpha_data.empty:
            alpha_data = ticker_df_2017
        else:
            alpha_data = pd.concat([alpha_data, ticker_df_2017], axis=1)
        # this API has a limit per minute
        time.sleep(0.8)
    except KeyboardInterrupt: break
    except Exception as e: 
        print(ticker, e)
### save ############################################
alpha_data.to_csv('data/data_prep/alpha_data.csv', index=True)
#####################################################
alpha_data.info(verbose=True)

  2%|▏         | 5/280 [00:05<03:57,  1.16it/s]

ABMD  'Time Series (Daily)'


 14%|█▎        | 38/280 [00:49<04:00,  1.01it/s]

BDX 'Time Series (Daily)'


 14%|█▍        | 39/280 [00:49<03:09,  1.27it/s]

...

FLT 'Time Series (Daily)'


 95%|█████████▍| 265/280 [06:04<00:17,  1.16s/it]

VIACOM 'Time Series (Daily)'


100%|██████████| 280/280 [06:25<00:00,  1.38s/it]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2017-01-03 to 2017-12-29
Data columns (total 273 columns):
 #    Column  Dtype  
---   ------  -----  
 0    MMM     float64
 1    AOS     float64
 2    ABT     float64
 3    ABBV    float64
 4    ATVI    float64
 5    ADBE    float64
 6    AAP     float64
 7    AMD     float64
 8    AES     float64
 9    AET     float64
 10   A       float64
 11   AKAM    float64
 12   ALK     float64
 13   ALXN    float64
 14   ALGN    float64
 15   ALLE    float64
 16   AGN     float64
 17   BFH     float64
 18   MO      float64
 19   AMZN    float64
 20   AEE     float64
 21   AXP     float64
 22   AMT     float64
 23   COR     float64
 24   AME     float64
 25   AMGN    float64
 26   ELV     float64
 27   AON     float64
 28   APA     float64
 29   ADM     float64
 30   T       float64
 31   ADSK    float64
 32   ADP     float64
 33   AVY     float64
 34   BALL    float64
 35   BAX     float64
 36   BKNG    float64
 37   BWA     flo

# MAEC dataset


[paper](https://dl.acm.org/doi/10.1145/3340531.3412879)
[GitHub](https://github.com/Earnings-Call-Dataset/MAEC-A-Multimodal-Aligned-Earnings-Conference-Call-Dataset-for-Financial-Risk-Prediction/tree/master)

In [ ]:
# Loop through the directory, each folder represents an earnings conference call; the folders are named as "Date_CompanyName".
MAEC_filename_data = []
for filename in os.listdir(MAEC_dir):
    date_str, ticker = filename.rsplit('_', 1)
    date_str = date_str.split('.')[0] 
    date = datetime.strptime(date_str, "%Y%m%d").strftime("%Y-%m-%d")
    MAEC_filename_data.append([ticker, date])
MAEC_filename_data = pd.DataFrame(MAEC_filename_data, columns=["Ticker", "Date"])
MAEC_filename_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3443 entries, 0 to 3442
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ticker  3443 non-null   object
 1   Date    3443 non-null   object
dtypes: object(2)
memory usage: 53.9+ KB


In [ ]:
MAEC_tickers = MAEC_filename_data.Ticker.unique().tolist()
MAEC_yahoo_data = yf.download(MAEC_tickers, start="2015-01-01", end="2018-12-31", group_by="ticker")
MAEC_yahoo_data.index = MAEC_yahoo_data.index.tz_localize(None)
successful_tickers = [col[0] if isinstance(col, tuple) else col for col in MAEC_yahoo_data.columns]
MAEC_yahoo_data.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in MAEC_yahoo_data.columns]
MAEC_yahoo_data = MAEC_yahoo_data.sort_index(axis=1)
MAEC_yahoo_data.info(verbose=True)
### save ############################################
MAEC_yahoo_data.to_csv('data/data_prep/MAEC_yahoo_data.csv', index=True)
#####################################################

[*********************100%***********************]  1213 of 1213 completed

249 Failed downloads:
['SIVB', 'AIR', 'GGP', 'HCN', 'ENDP', 'CBG', 'GOV', 'ACXM', 'QSII', 'PDCE', 'BOBE']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2015-01-01 -> 2018-12-31)')
['KRA', 'MTSC', 'CLI', 'COG', 'WIRE', 'DSPG', 'VRTV', 'SPN', 'QEP', 'WTR', 'MCF', 'MGLN', 'WWE', 'ACC', 'MDP', 'UIHC', 'PLT', 'FB', 'SFLY', 'TIVO', 'VSI', 'DPLO', 'UTX', 'CMO', 'BBBY', 'CRZO', 'TSS', 'RCII', 'HT', 'IVC', 'ERA', 'BRS', 'CTL', 'DRE', 'MBFI', 'MANT', 'AKRX', 'MON', 'MNK', 'RTN', 'RUTH', 'DF', 'TMST', 'HRC', 'OFC', 'NAVG', 'SAFM', 'UNT', 'ITG', 'XLNX', 'DNKN', 'EBIX', 'ESL', 'CXO', 'CY', 'QHC', 'GHL', 'FLIR', 'LPT', 'ATVI', 'FRAN', 'ARRS', 'ESV', 'POL', 'CBB', 'IIVI', 'EXTN', 'STMP', 'MNTA', 'MDSO', 'DISCA', 'CDR', 'AKS', 'INT', 'PBCT', 'CRY', 'RECN', 'DSW', 'CPE', 'TIF', 'BCOR', 'PSB', 'BID', 'CLD', 'CHFC', 'TYPE', 'TECD', 'CTB', 'AEGN', 'CHS', 'WDR', 'VIAB', 'AAWW', 'HMSY', 'AEL', 'SJI', 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1005 entries, 2015-01-02 to 2018-12-28
Data columns (total 7278 columns):
 #     Column           Dtype  
---    ------           -----  
 0     AAN_Adj Close    float64
 1     AAN_Close        float64
 2     AAN_High         float64
 3     AAN_Low          float64
 4     AAN_Open         float64
 5     AAN_Volume       float64
 6     AAOI_Adj Close   float64
 7     AAOI_Close       float64
 8     AAOI_High        float64
 9     AAOI_Low         float64
 10    AAOI_Open        float64
 11    AAOI_Volume      int64  
 12    AAON_Adj Close   float64
 13    AAON_Close       float64
 14    AAON_High        float64
 15    AAON_Low         float64
 16    AAON_Open        float64
 17    AAON_Volume      int64  
 18    AAPL_Adj Close   float64
 19    AAPL_Close       float64
 20    AAPL_High        float64
 21    AAPL_Low         float64
 22    AAPL_Open        float64
 23    AAPL_Volume      int64  
 24    AAP_Adj Close    float64
 25    AA

In [ ]:
print(len(MAEC_tickers))
print(len(pd.unique(successful_tickers)))

1213
1213


In [ ]:
MAEC_alpha_data = pd.DataFrame()
bad_tickers = []
tickers = MAEC_filename_data.Ticker.unique().tolist()
for ticker in tqdm(tickers):
    try:
        response = requests.get(alpha_url.format(ticker,API_KEY)).json()
        ticker_df = pd.DataFrame(response['Time Series (Daily)']).T
        ticker_df.columns = [f"{ticker}_{col}" for col in alpha_columns]
        ticker_df.index = pd.to_datetime(ticker_df.index)
        ticker_df = ticker_df.apply(pd.to_numeric, errors='coerce')
        ### save ############################################
        ticker_df.to_csv(alpha_dir.format(ticker), index=True)
        #####################################################
        ticker_df_2017 = ticker_df[(ticker_df.index.year >= 2015) & (ticker_df.index.year <= 2018)]
        ticker_df_2017 = ticker_df_2017.rename(columns={f"{ticker}_Adj Close": ticker})
        ticker_df_2017 = ticker_df_2017[[ticker]].copy()
        if MAEC_alpha_data.empty:
            MAEC_alpha_data = ticker_df_2017
        else:
            MAEC_alpha_data = pd.concat([MAEC_alpha_data, ticker_df_2017], axis=1)
        # this API has a limit per minute
        time.sleep(0.8)
    except KeyboardInterrupt: break
    except Exception as e: 
        bad_tickers.append([ticker, e])

MAEC_alpha_data.info(verbose=True)
### save ############################################
MAEC_alpha_data.to_csv('data/data_prep/MAEC_alpha_data.csv', index=True)
#####################################################

100%|██████████| 1213/1213 [28:50<00:00,  1.43s/it]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2015-01-02 to 2018-12-31
Data columns (total 1155 columns):
 #     Column  Dtype  
---    ------  -----  
 0     LMAT    float64
 1     ACIW    float64
 2     ADSK    float64
 3     AMWD    float64
 4     ANSS    float64
 5     CRI     float64
 6     ECPG    float64
 7     IRDM    float64
 8     LYV     float64
 9     ORN     float64
 10    NWN     float64
 11    RDC     float64
 12    SJI     float64
 13    BID     float64
 14    MDR     float64
 15    TG      float64
 16    AZO     float64
 17    NX      float64
 18    AMED    float64
 19    ANF     float64
 20    ESL     float64
 21    FL      float64
 22    KOPN    float64
 23    RAVN    float64
 24    PLCE    float64
 25    ULTA    float64
 26    BKE     float64
 27    RGEN    float64
 28    DSW     float64
 29    FDS     float64
 30    GIII    float64
 31    PAYX    float64
 32    REX     float64
 33    SCHL    float64
 34    CCL     float64
 35    MOV     float64

In [ ]:
errors = [str(pair[1]) for pair in bad_tickers]
bad_tickers = [pair[0] for pair in bad_tickers]
print(len(bad_tickers))
print(np.unique(errors))

58
["'Time Series (Daily)'" 'Expecting value: line 1 column 1 (char 0)']


In [ ]:
print(bad_tickers)

['GPS', 'JCP', 'TUP', 'BBT', 'MDP', 'LL', 'ABC', 'PKI', 'HFC', 'HSC', 'CBB', 'ILG', 'JCOM', 'EBIX', 'ENDP', 'BIG', 'ASNA', 'IVC', 'BCOR', 'INT', 'FRED', 'CAMP', 'ELY', 'COG', 'CLD', 'CRY', 'PLT', 'FRAN', 'ADS', 'CPSI', 'FBHS', 'CHFC', 'UIHC', 'OFC', 'TMST', 'FTD', 'SWM', 'WPG', 'WLTW', 'AKRX', 'BLL', 'DF', 'TLRD', 'SPN', 'CLI', 'ESV', 'RCII', 'ANTM', 'RE', 'NCR', 'NEWM', 'PEI', 'LCI', 'ERA', 'ACOR', 'FB', 'AAXN', 'NLS']
